In [1]:
import pandas as pd

In [22]:
df = pd.read_csv("data/karlskrona/smhi-lufttemperatur.csv", sep=";", index_col=["Datum"], parse_dates=True)

In [24]:
df

,Tid (UTC),Lufttemperatur,Kvalitet
Datum,,,
2010-03-01,00:00:00,2.0,G
2010-03-01,01:00:00,2.2,Y
2010-03-01,02:00:00,2.2,Y
2010-03-01,03:00:00,1.7,G
2010-03-01,04:00:00,1.7,Y
...,...,...,...
2022-08-01,02:00:00,16.2,G
2022-08-01,03:00:00,16.0,G
2022-08-01,04:00:00,16.1,G


### Data preprocessing

### Data exploration

### 1. Choose predicted variable

### 2. Choose predictors 

### 3. Split data

### 4. Building model

### 5. Evaluating model effiency

### 6. Evaluating model accuracy